In [1]:
%pwd

'/home/sarath_kumar/ImagoAI/notebook'

In [ ]:
# import os
# os.chdir("../")
# %pwd

'/home/sarath_kumar/ImagoAI'

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, Dropout

2025-03-12 22:12:25.943441: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-12 22:12:25.952026: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-12 22:12:25.981844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741797746.030173  396525 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741797746.043788  396525 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 22:12:26.098949: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [10]:

# Load dataset
df = pd.read_csv("data/TASK-ML-INTERN.csv")  # Replace with actual dataset
df.drop("hsi_id",axis=1,inplace = True)


# Remove outliers using IQR
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

# Separate features and target
X = df.iloc[:, :-1].values  # All except last column
y = df.iloc[:, -1].values   # Last column (target variable)

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=2,whiten=True,svd_solver="randomized")
X_pca = pca.fit_transform(X_scaled)

# Variance Explained
print("Explained Variance Ratio:", pca.explained_variance_ratio_)

# Reshape data for CNN and LSTM (samples, time steps, features)
X_reshaped = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

# Train-Test Split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)


Explained Variance Ratio: [0.86843952 0.05101268]


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Build LSTM Model
lstm_model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(1, X_train.shape[2])),  # Correct input shape
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Regression output
])

# Compile Model
lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train Model
lstm_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))




Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 130ms/step - loss: 1103225.7500 - mae: 628.9891 - val_loss: 940833.9375 - val_mae: 584.7264
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 1308428.3750 - mae: 696.3333 - val_loss: 939389.4375 - val_mae: 584.1230
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 1178659.2500 - mae: 661.6633 - val_loss: 937253.7500 - val_mae: 583.2274
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 1167084.0000 - mae: 666.3539 - val_loss: 934188.3125 - val_mae: 581.9439
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 1230763.1250 - mae: 691.9442 - val_loss: 930095.0625 - val_mae: 580.2354
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 1212452.2500 - mae: 680.3312 - val_loss: 924906.3750 - val_mae: 578.1444
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 1092936.1250 - mae: 642.7353 - val_loss: 918921.9375 - val_mae: 575.7702
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 1216922.0000 - m

In [13]:
# Predictions
y_pred = lstm_model.predict(X_test)

# Evaluate Model
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R² Score: {r2:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
MAE: 513.0575, RMSE: 781.5565, R² Score: -0.0190
